# Vector Similarity Astra-Bedrock Search QA Quickstart

Set up a simple Question-Answering system with LangChain and Amazon Bedrock, using Astra DB as the Vector Database.

## Prerequisites

Make sure you have a vector-capable Astra database (get one for free at [astra.datastax.com](https://astra.datastax.com)):

- Astra DB
    - an **Access Token** for your database with role _Database Administrator_ (see [here](https://awesome-astra.github.io/docs/pages/astra/create-token/) for details).
    
- Amazon Web Services
    - an Identity with access to **Amazon Bedrock**.

## Import needed libraries

In [1]:
import json
import os
import sys
from getpass import getpass
import warnings
import boto3
import botocore

from io import StringIO
import sys
import textwrap
import os
from typing import Optional

# External Dependencies:
import boto3
from botocore.config import Config

warnings.filterwarnings('ignore')

import boto3
import cassio
from litellm import completion
from langchain.embeddings import BedrockEmbeddings
from langchain.llms import Bedrock
from langchain.vectorstores import Cassandra
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader

## Astra DB Setup

In [2]:
from dotenv import load_dotenv

load_dotenv()

## set ENV variables
os.environ["OPENAI_API_KEY"] = "openai key"
os.environ["COHERE_API_KEY"] = "cohere key"

ASTRA_DB_ID = os.environ["ASTRA_DB_ID"]
ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_VECTOR_ENDPOINT = os.environ["ASTRA_VECTOR_ENDPOINT"]
#ASTRA_DB_KEYSPACE = "blueillusion"
#ASTRA_DB_COLLECTION = "sydshakespeare"
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_PROJECT"] = "blueillusion"
#os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [3]:
cassio.init(
    token=ASTRA_DB_APPLICATION_TOKEN,
    database_id=ASTRA_DB_ID,
)

## AWS Credentials Setup

These are set as environment variables for usage by the subsequent `boto3` calls. Please refer to [boto3's documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html) on the possible ways to supply your credentials in a more production-like environment.

In [ ]:
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]

In [4]:
def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))
        



def get_bedrock_client(
    assumed_role: Optional[str] = None,
    region: Optional[str] = None,
    runtime: Optional[bool] = True,
):
    """Create a boto3 client for Amazon Bedrock, with optional configuration overrides

    Parameters
    ----------
    assumed_role :
        Optional ARN of an AWS IAM role to assume for calling the Bedrock service. If not
        specified, the current active credentials will be used.
    region :
        Optional name of the AWS Region in which the service should be called (e.g. "us-east-1").
        If not specified, AWS_REGION or AWS_DEFAULT_REGION environment variable will be used.
    runtime :
        Optional choice of getting different client to perform operations with the Amazon Bedrock service.
    """
    if region is None:
        target_region = os.environ.get("AWS_REGION", os.environ.get("AWS_DEFAULT_REGION"))
    else:
        target_region = region

    print(f"Create new client\n  Using region: {target_region}")
    session_kwargs = {"region_name": target_region}
    client_kwargs = {**session_kwargs}

    profile_name = os.environ.get("AWS_PROFILE")
    if profile_name:
        print(f"  Using profile: {profile_name}")
        session_kwargs["profile_name"] = profile_name

    retry_config = Config(
        region_name=target_region,
        retries={
            "max_attempts": 10,
            "mode": "standard",
        },
    )
    session = boto3.Session(**session_kwargs)

    if assumed_role:
        print(f"  Using role: {assumed_role}", end='')
        sts = session.client("sts")
        response = sts.assume_role(
            RoleArn=str(assumed_role),
            RoleSessionName="langchain-llm-1"
        )
        print(" ... successful!")
        client_kwargs["aws_access_key_id"] = response["Credentials"]["AccessKeyId"]
        client_kwargs["aws_secret_access_key"] = response["Credentials"]["SecretAccessKey"]
        client_kwargs["aws_session_token"] = response["Credentials"]["SessionToken"]

    if runtime:
        service_name='bedrock-runtime'
    else:
        service_name='bedrock'

    bedrock_client = session.client(
        service_name=service_name,
        config=retry_config,
        **client_kwargs
    )

    print("boto3 Bedrock client successfully created!")
    print(bedrock_client._endpoint)
    return bedrock_client






# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."




## Set up AWS Bedrock objects

In [5]:

bedrock_runtime = get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region='us-east-1' #os.environ.get("AWS_DEFAULT_REGION", None)
)
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1",
    client=bedrock_runtime
    )

Create new client
  Using region: us-east-1
  Using profile: AWSAdministratorAccess-590312749310
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## Set up the Vector Store

This command will create a suitable table in your database if it does not exist yet:

In [6]:
vector_store = Cassandra(
    embedding=bedrock_embeddings,
    table_name="sydshakespeare",
    session=None,  # <-- meaning: use the global defaults from cassio.init()
    keyspace=None,  # <-- meaning: use the global defaults from cassio.init()
)

## Populate the database

Add lines for the text of "Romeo and Astra", Scene 5, Act 3

In [7]:
# retrieve the text of a scene from act 5 of Romeo and Astra. 
# Juliet's name was changed to Astra to prevent the LLM from "cheating" when providing an answer.
! mkdir -p "texts"
! curl "https://raw.githubusercontent.com/awesome-astra/docs/main/docs/pages/aiml/aws/bedrock_resources/romeo_astra.json" \
    --output "texts/romeo_astra.json"
input_lines = json.load(open("texts/romeo_astra.json"))

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file texts already exists.
Error occurred while processing: texts.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 75985  100 75985    0     0   212k      0 --:--:-- --:--:-- --:--:--  213k


Next, you'll populate the database with the lines from the play.
This can take a couple of minutes, please be patient.  In total there are 321 lines.


In [8]:
input_documents = []

for input_line in input_lines:
    if (input_line["ActSceneLine"] != ""):
        (act, scene, line) = input_line["ActSceneLine"].split(".")
        location = "Act {}, Scene {}, Line {}".format(act, scene, line)
        metadata = {"act": act, "scene": scene, "line": line}
    else:
        location = ""
        metadata = {}
    quote_input = "{} : {} : {}".format(location, input_line["Player"], input_line["PlayerLine"])
    input_document = Document(page_content=quote_input, metadata=metadata)
    input_documents.append(input_document)
    
print(f"Adding {len(input_documents)} documents ... ", end="")
vector_store.add_documents(documents=input_documents, batch_size=50)
print("Done.")

Adding 321 documents ... Done.


## Answer questions

In [9]:
prompt_template_str = """Human: Use the following pieces of context to provide a concise answer to the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

<context>
{context}
</context

Question: {question}

Assistant:"""

prompt = PromptTemplate.from_template(prompt_template_str)

We choose to use the following LLM model (see [this page](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html#model-parameters-general) for more info):

In [10]:
model_id = "anthropic.claude-v2"

Here the question-answering function is set up, implementing the RAG pattern:

In [11]:
req_accept = "application/json"
req_content_type = "application/json"

# This, created from the vector store, will fetch the top relevant documents given a text query
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

def answer_question(question: str, verbose: bool = False) -> str:
    if verbose:
        print(f"\n[answer_question] Question: {question}")
    # Retrieval of the most relevant stored documents from the vector store:
    context_docs = retriever.get_relevant_documents(question)
    context = "\n".join(doc.page_content for doc in context_docs)
    if verbose:
        print("\n[answer_question] Context:")
        print(context)
    # Filling the prompt template with the current values
    llm_prompt_str = prompt.format(
        question=question,
        context=context,
    )
    # Invocation of the Amazon Bedrock LLM for text completion -- ultimately obtaining the answer
    llm_body = json.dumps({"prompt": llm_prompt_str, "max_tokens_to_sample": 500})
    llm_response = bedrock_runtime.invoke_model(
        body=llm_body,
        modelId=model_id,
        accept=req_accept,
        contentType=req_content_type,
    )
    llm_response_body = json.loads(llm_response["body"].read())
    answer = llm_response_body["completion"].strip()
    if verbose:
        print(f"\n[answer_question] Answer: {answer}\n")
    return answer

In [15]:
my_answer = answer_question("Who dies in the story?")
print("=" * 60)

print(my_answer)

Based on the provided context, it seems that Astra and Tybalt are characters who die in the story. The lines mention Astra being found dead, bleeding and warm, and refer to Tybalt's "untimely death". The Prince also says someone came to the vault to die and lie with Astra, implying she is dead. So Astra and Tybalt are two characters who die, according to this context.


Let's take a look at the RAG process piece-wise:

In [13]:
my_answer = answer_question("Who dies in the story?", verbose=True)
print("=" * 60)
print(my_answer)


[answer_question] Question: Who dies in the story?

[answer_question] Context:
Act 5, Scene 3, Line 184 : First Watchman : And Astra bleeding, warm, and newly dead,
Act 5, Scene 3, Line 184 : First Watchman : And Astra bleeding, warm, and newly dead,
Act 5, Scene 3, Line 244 : FRIAR LAURENCE : Was Tybalt's dooms-day, whose untimely death
Act 5, Scene 3, Line 300 : PRINCE : Came to this vault to die, and lie with Astra.
Act 5, Scene 3, Line 300 : PRINCE : Came to this vault to die, and lie with Astra.

[answer_question] Answer: Based on the provided context, it seems that Astra and Tybalt are characters who die in the story. The lines mention Astra being found dead, and refer to Tybalt's "untimely death".

Based on the provided context, it seems that Astra and Tybalt are characters who die in the story. The lines mention Astra being found dead, and refer to Tybalt's "untimely death".


### Interactive QA session

In [17]:
user_question = ""
while True:
    user_question = input("Enter a question (empty to quit):").strip()
    if user_question:
        print(f"Answer ==> {answer_question(user_question)}")
    else:
        print("[User, AI exeunt]")
        break

Answer ==> Based on the context provided, Juliet is Romeo's faithful wife who is found dead and appears to have committed violence on herself.
Answer ==> Based on the provided context, I don't know who Romeo is referring to in these lines. The context provided does not give enough information to determine who Romeo is talking about.
Answer ==> Based on the provided context, I do not know who "tyblah" is. The context includes lines spoken by the Prince, Page, and Paris, but does not mention anyone named "tyblah". Without more context about this character, I cannot determine who they are.
Answer ==> Based on the context provided, Tybalt is a character who was killed in an "untimely death" prior to the events described in the lines from Friar Laurence. Friar Laurence mentions that Astra pined for someone other than Tybalt, implying that Tybalt is a separate individual who is now deceased.
[User, AI exeunt]


## Additional resources

To learn more about Amazon Bedrock, visit this page: [Introduction to Amazon Bedrock](https://github.com/aws-samples/amazon-bedrock-samples/tree/main/introduction-to-bedrock).